In [1]:
import torch
from torchvision.ops import roi_pool
import numpy as np

## 手动创建一张feature map

In [2]:
feature_maps = (torch.rand(1, 6, 10, 10) * 100).to(torch.uint8).float()
roi = torch.rand(1, 4) * 20

In [3]:
print(f'{feature_maps[0][0]}:.2f')

tensor([[27.,  2., 34., 29., 70., 96., 72.,  5., 17., 78.],
        [22., 71., 38.,  1., 23., 97., 68., 36., 36., 69.],
        [98., 59., 81., 87., 79.,  6., 43., 55., 10., 22.],
        [53.,  0.,  4., 77., 27., 48., 99., 55., 29., 50.],
        [50.,  8., 88., 86., 58., 11., 29., 89., 65., 72.],
        [69., 58., 81., 48., 94., 32., 37., 15., 93., 55.],
        [ 3., 74., 57.,  4., 67., 48., 83., 45., 69.,  2.],
        [12.,  1., 48., 93., 38., 63., 95., 42., 48., 74.],
        [12., 34.,  7., 68., 94.,  5., 70., 60., 20.,  3.],
        [90., 72., 99., 58., 37., 40., 99., 91., 59., 70.]]):.2f


In [4]:
feature_maps[0][0] = torch.zeros(10, 10)
feature_maps[0][0][0][5] = 10
feature_maps[0][0][3][0] = 0
feature_maps[0][0]

tensor([[ 0.,  0.,  0.,  0.,  0., 10.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

## 创建roi kernel

In [5]:
roi = torch.tensor([1, 1, 70, 70], dtype=torch.float32)[None, :]

## torch roipooling

In [6]:
# roi = torch.rand(1, 4) * 20
pool = roi_pool(feature_maps, [roi], (3,3), 1/10)
pool[0][0]

tensor([[ 0., 10., 10.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]])

## 使用numpy实现roipooling

In [7]:
import numpy as np
import math

def simulate_roipool(feature, roi, output_size, spatial_scale):
    # roi format: [xmin, ymin, xmax, ymax]
    assert isinstance(feature, np.ndarray)
    assert isinstance(roi, np.ndarray)
    assert feature.ndim == 2
    
    roi = np.floor(roi)
    roi = np.floor(roi * spatial_scale)
    xmin, ymin, xmax, ymax = roi.astype(np.int32)
    height = ymax - ymin + 1
    width = xmax - xmin + 1
    roi_mat = feature[ymin:ymax, xmin:xmax]
    pool = []
    
    for i in range(output_size[1]):
        for j in range(output_size[0]):
            x_start = math.floor(j * (width / output_size[0]))
            x_end = math.ceil((j+1) * (width / output_size[0]))
            y_start = math.floor(i * (height / output_size[1]))
            y_end = math.ceil((i+1) * (height / output_size[1]))
            bin_ = roi_mat[y_start:y_end, x_start:x_end]
            pool.append(np.max(bin_))
            
    return np.asarray(pool).reshape(output_size)

In [8]:
a = feature_maps[0][0].numpy()
b = np.array([1, 1, 70, 70])

pool = simulate_roipool(a, b, (3,3), 1/10)
pool

array([[ 0., 10., 10.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]], dtype=float32)